## Person re-identification using Siamese Network

<b> Why Siamese Network ?</b>

A Siamese Network is a type of neural network architecture that is used for tasks that involve finding similarities or differences between two input samples. 

The network is trained to output a high value when the two input samples are similar and a low value when they are dissimilar. This makes it useful for a variety of applications, such as image or text similarity matching, face recognition, and signature verification.

One of main advantages of a Siamese Network is that it can be trained with very few examples, making it useful for applications where data is limited. Additionally, the shared weights between the two subnetworks allow the model to generalize well to new inputs.

We will see the applications for re-identification of images in this notebook.



About Dataset 

Dataset consists of Anchor image. positive image that is closer to the anchor image, and negative image which is farther apart. 

Each of the row will pass through siamese neural net for example resnet, and will output embeddings. These are then passed to a triplet loss function, and we learn the feature representation.  The reqiuired output will be postvbe amd anchor images near to each other and negative should be far apart.

In [ ]:
!pip install segmentation-models-pytorch
!pip install -U git+https://github.com/albumentations-team/albumentations
!pip install --upgrade opencv-contrib-python

<img src= "/Users/dhwaninijhawan/Documents/Portfolio/DeepLearning/siamese.png">

Import necessary libraries

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import timm
import sys
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader

from skimage import io
from sklearn.model_selection import train_test_split

from tqdm import tqdm

sys.path.append('/content/Person-Re-Id-Dataset')

In [12]:
data_dir = '/Users/dhwaninijhawan/Documents/Portfolio/DeepLearning/Person-Re-Id-Dataset/train'
csv_file = '/Users/dhwaninijhawan/Documents/Portfolio/DeepLearning/Person-Re-Id-Dataset/train.csv'
batch_size = 32
learning_rate = 0.001
epochs = 15
device = 'cuda'

In [6]:
!git clone https://github.com/parth1620/Person-Re-Id-Dataset

Cloning into 'Person-Re-Id-Dataset'...
remote: Enumerating objects: 12942, done.
remote: Counting objects: 100% (12942/12942), done.
remote: Compressing objects: 100% (12942/12942), done.jects:  87% (11260/12942)
remote: Total 12942 (delta 0), reused 12942 (delta 0), pack-reused 0
Receiving objects: 100% (12942/12942), 27.68 MiB | 3.79 MiB/s, done.
Updating files: 100% (12939/12939), done.


In [11]:
df = pd.read_csv(csv_file)
df.head()

,Anchor,Negative,Positive
0,1420_c5s3_052165_01.jpg,1334_c6s3_061492_05.jpg,1420_c3s3_051678_01.jpg
1,1420_c3s3_061978_03.jpg,0234_c3s3_079494_02.jpg,1420_c6s3_085567_02.jpg
2,1420_c5s3_062565_05.jpg,0475_c2s1_122816_08.jpg,1420_c3s3_051653_01.jpg
3,1420_c6s3_085592_04.jpg,0662_c2s2_036662_05.jpg,1420_c1s6_013446_04.jpg
4,0663_c5s3_085987_03.jpg,1463_c2s3_098102_02.jpg,0663_c3s3_085544_06.jpg
